In [1]:
import os
import pandas as pd
import numpy as np
import cv2  # conda install -c conda-forge opencv
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams['axes.grid'] = False
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False
plt.rcParams["font.size"] = 15
plt.rcParams["savefig.bbox"] = "tight"
plt.rcParams['figure.facecolor'] = 'white'

### Gather info on data here
- 2024_0312_200031_053.MP4 - 1:14-29 (ext from contraction), after period of inactivity, whirling without shaking - 1:45-1:49 (end, not needed for now, maybe for vortex)
- 2024_0312_200250_054.MP4, 0-14 (ext from contract), 1:25-35 (retraction and before), 1:58-2:08 (ext from contract), half disc shown from time to time
- 2024_0312_200610_055.MP4, 60fps, D_high, 0:32-47 (extention from contraction), 1:16-26 (retraction), 2:01-11 (extention after half-retraction of half-corona, can be left out for now too> take entirely as that sequence of events!)
- get fps, its important for corona things...

- 2024_0312_201213_057.MP4, 2:04-9, ret-elongate-ext (Dlow)
- 2024_0312_201502_058.MP4, 33-39, ret-elongate-ext  (Dlow)
- 2024_0312_084233_028.MP4 # (2:05-2:15, slow extent)
- 2024_0311_153344_021.MP4 - vgood 7min, 0-5 (ext), 18-28 (ret-ext), 28-38 (ret-probe-move), 1:15-19 (ret-probe-move), 6:54-6:56 (ret-probe),
- 2024_0311_153013_020.MP4 - 2:57-3:02 (ret-probe-ext)
- 2024_0311_152452_018.MP4 - 0-15 (ret-probe-ext, move-probe-ext-ret-ext), 43-47 (ret-move), 
- 2024_0311_152225_017.MP4, move while whirling, great vortex


In [2]:

"""
# extent from contraction
"2024_0312_200031_053.MP4" 74,89
"2024_0312_200250_054.MP4" 0,14 & 118,128
"2024_0312_200610_055.MP4" 32,42 & 121,131 # (its at 60fps)
"2024_0312_084233_028.MP4" 125, 135# (2:05-2:15, slow extent)
"2024_0311_153344_021.MP4", 0,5
# retraction
"2024_0312_200250_054.MP4" 85,95
"2024_0312_200610_055.MP4" 76,86  # valuable @60fps
"2024_0311_153344_021.MP4" 414,417
# ret-elongate-ext
"2024_0312_201213_057.MP4" 124,129
"2024_0312_201502_058.MP4" 30,39
"2024_0311_153344_021.MP4" 18,28
# ret-probe-move
"2024_0311_153344_021.MP4" 28,38, 75,79
"2024_0311_152452_018.MP4" 43,47
# ret-probe-ext
"2024_0311_153013_020.MP4" 177,182
# extent-retract-extent
"2024_0312_200610_055.MP4" 121,131
# ret-probe-ext, move-probe-ext-ret-ext
"2024_0311_152452_018.MP4" 0,15
"""

'\n# extent from contraction\n"2024_0312_200031_053.MP4" 74,89\n"2024_0312_200250_054.MP4" 0,14 & 118,128\n"2024_0312_200610_055.MP4" 32,42 & 121,131 # (its at 60fps)\n"2024_0312_084233_028.MP4" 125, 135# (2:05-2:15, slow extent)\n"2024_0311_153344_021.MP4", 0,5\n# retraction\n"2024_0312_200250_054.MP4" 85,95\n"2024_0312_200610_055.MP4" 76,86  # valuable @60fps\n"2024_0311_153344_021.MP4" 414,417\n# ret-elongate-ext\n"2024_0312_201213_057.MP4" 124,129\n"2024_0312_201502_058.MP4" 30,39\n"2024_0311_153344_021.MP4" 18,28\n# ret-probe-move\n"2024_0311_153344_021.MP4" 28,38, 75,79\n"2024_0311_152452_018.MP4" 43,47\n# ret-probe-ext\n"2024_0311_153013_020.MP4" 177,182\n# extent-retract-extent\n"2024_0312_200610_055.MP4" 121,131\n# ret-probe-ext, move-probe-ext-ret-ext\n"2024_0311_152452_018.MP4" 0,15\n'

In [3]:
tmp_dict = {
    1 : ["2024_0312_200031_053", 74, 89],
    2 : ["2024_0312_200250_054", 0, 14],
    3 : ["2024_0312_200250_054", 118, 128],
    4 : ["2024_0312_200610_055", 32, 42],
    5 : ["2024_0312_200610_055", 121, 131],
    6 : ["2024_0312_084233_028", 125, 135],
    7 : ["2024_0311_153344_021", 0, 5],
    8 : ["2024_0312_200250_054", 85, 95],
    9 : ["2024_0312_200610_055", 76, 86],
    10: ["2024_0311_153344_021", 414, 417],
    11: ["2024_0312_201213_057", 124, 129],
    12: ["2024_0312_201502_058", 30, 39],
    13: ["2024_0311_153344_021", 18, 28],
    14: ["2024_0311_153344_021", 28, 38],
    15: ["2024_0311_153344_021", 74, 79],
    16: ["2024_0311_152452_018", 43, 47],
    17: ["2024_0311_153013_020", 177, 182],
    18: ["2024_0312_200610_055", 121, 131],
    19: ["2024_0311_152452_018", 0, 15],
}
# idx 1-7 = extent from contraction; 8-10 = retraction; 11-13 = ret-elongate-ext; 14-16 = ret-probe-move; 17 = ret-probe-ext; 18 = extent-retract-extent; 19 = ret-probe-ext, move-probe-ext-ret-ext
# have another CELL for CHECKING THE CROP
df = pd.DataFrame.from_dict(tmp_dict, orient="index")
df.columns = ["filename", "tstart", "tend"]
df["desc"] = str(0)
df.loc[:7, "desc"] = "extent from contraction"
df.loc[8:10, "desc"] = "retraction"
df.loc[11:13, "desc"] = "ret-elongate-ext"
df.loc[14:16, "desc"] = "ret-probe-move"
df.loc[17, "desc"] = "ret-probe-ext"
df.loc[18, "desc"] = "extent-retract-extent"
df.loc[19, "desc"] = "ret-probe-ext, move-probe-ext-ret-ext"

file_ext = ".MP4"
vid_dir = "moss"
vid_path = f"./data/videos/{vid_dir}/"

fps, width, height = [], [], []
for nth, row in df.iterrows():
    vid_filename = row["filename"] + file_ext
    vid_filepath = f"{vid_path}{vid_filename}"
    video = cv2.VideoCapture(vid_filepath)
    fps.append(int(video.get(cv2.CAP_PROP_FPS)))
    width.append(int(video.get(cv2.CAP_PROP_FRAME_WIDTH)))
    height.append(int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    video.release()

df["fps"] = fps
df["width"] = width
df["height"] = height
df["duration"] = df.tend - df.tstart
df#df = df.sort_values(by="filename")

,filename,tstart,tend,desc,fps,width,height,duration
1,2024_0312_200031_053,74,89,extent from contraction,24,2880,2160,15
2,2024_0312_200250_054,0,14,extent from contraction,24,2880,2160,14
3,2024_0312_200250_054,118,128,extent from contraction,24,2880,2160,10
4,2024_0312_200610_055,32,42,extent from contraction,60,1920,1080,10
5,2024_0312_200610_055,121,131,extent from contraction,60,1920,1080,10
6,2024_0312_084233_028,125,135,extent from contraction,24,2880,2160,10
7,2024_0311_153344_021,0,5,extent from contraction,24,2880,2160,5
8,2024_0312_200250_054,85,95,retraction,24,2880,2160,10
9,2024_0312_200610_055,76,86,retraction,60,1920,1080,10
10,2024_0311_153344_021,414,417,retraction,24,2880,2160,3


In [4]:
from moviepy.editor import VideoFileClip
from moviepy.editor import *

In [5]:
# the following manually noted cropping points:
df["xy"] = [
    [250,1250,0,1100],
    [250,1250,0,1100],
    [250,1250,0,1100],
    [200,850,0,600],
    [200,850,0,600],
    [800,1800,500,1500],
    [1200,1750,1000,2000],
    [250,1250,0,1100],
    [200,850,0,600],
    [800,1900,650,1400],
    [1000,1500,0,1000],
    [750,1100,0,600],
    [1200,1800,850,2000],
    [1050,1800,800,2000],
    [600,1800,0,1300],
    [1200,2200,800,1600],
    [1050,1750,900,2000],
    [200,850,0,600],
    [700,2100,400,1800]
    ]

df[["x1", "x2", "y1", "y2"]] = df["xy"].apply(lambda x: pd.Series(x))
df = df.drop('xy', axis=1, inplace=False)

In [6]:
vid_save_filenames = []
for nth, row in df.iterrows():
    ts = row["tstart"]
    te = row["tend"]
    trim_duration = te - ts
    x1, x2, y1, y2 = row["x1"], row["x2"], row["y1"], row["y2"]
    vid_save_filename = f"{row['filename']}_trimmed_{trim_duration}sec_ts{ts}-te{te}_cropped_{x1}-{x2}-{y1}-{y2}.mp4"
    vid_save_filenames.append(vid_save_filename)

df["save_filename"] = vid_save_filenames
SAVE_DF = False
if SAVE_DF: df.to_csv("./data/videos/moss/moss-corona-dataset.csv", index=False)
df

,filename,tstart,tend,desc,fps,width,height,duration,x1,x2,y1,y2,save_filename
1,2024_0312_200031_053,74,89,extent from contraction,24,2880,2160,15,250,1250,0,1100,2024_0312_200031_053_trimmed_15sec_ts74-te89_c...
2,2024_0312_200250_054,0,14,extent from contraction,24,2880,2160,14,250,1250,0,1100,2024_0312_200250_054_trimmed_14sec_ts0-te14_cr...
3,2024_0312_200250_054,118,128,extent from contraction,24,2880,2160,10,250,1250,0,1100,2024_0312_200250_054_trimmed_10sec_ts118-te128...
4,2024_0312_200610_055,32,42,extent from contraction,60,1920,1080,10,200,850,0,600,2024_0312_200610_055_trimmed_10sec_ts32-te42_c...
5,2024_0312_200610_055,121,131,extent from contraction,60,1920,1080,10,200,850,0,600,2024_0312_200610_055_trimmed_10sec_ts121-te131...
6,2024_0312_084233_028,125,135,extent from contraction,24,2880,2160,10,800,1800,500,1500,2024_0312_084233_028_trimmed_10sec_ts125-te135...
7,2024_0311_153344_021,0,5,extent from contraction,24,2880,2160,5,1200,1750,1000,2000,2024_0311_153344_021_trimmed_5sec_ts0-te5_crop...
8,2024_0312_200250_054,85,95,retraction,24,2880,2160,10,250,1250,0,1100,2024_0312_200250_054_trimmed_10sec_ts85-te95_c...
9,2024_0312_200610_055,76,86,retraction,60,1920,1080,10,200,850,0,600,2024_0312_200610_055_trimmed_10sec_ts76-te86_c...
10,2024_0311_153344_021,414,417,retraction,24,2880,2160,3,800,1900,650,1400,2024_0311_153344_021_trimmed_3sec_ts414-te417_...


# Display frames so cropping can be determined

In [7]:
SHOW_FRAMES = False
SAVE_VIDEO = False
#correct_videos_indices = [10,14,15,17,19] > simply go over every entry

#for index, row in df.iloc[correct_videos_indices].iterrows():
for nth, row in df.iterrows():  # iloc[18:19].
    
    vid_filename = row["filename"] + file_ext
    vid_filepath = f"{vid_path}{vid_filename}"
    ts = row["tstart"]
    te = row["tend"]
    trim_duration = te - ts

    # Video Trimming
    video_clip = VideoFileClip(vid_filepath).subclip(ts, te)

    # Show frames only to determine what to crop
    if SHOW_FRAMES:
        show_frames = []
        show_frames.append(video_clip.get_frame(t=0))
        show_frames.append(video_clip.get_frame(t=int(trim_duration/2)))
        show_frames.append(video_clip.get_frame(t=-1))

        fig, ax = plt.subplots(1, len(show_frames), figsize=(20,20))
        for ith, frame in enumerate(show_frames):
            ax[ith].imshow(frame)
        ax[1].set_title("idx : " + str(nth) + " - " + vid_filename)
    
    # Video Cropping
    if SAVE_VIDEO:
        print(f"SAVING TRIMMED AND CROPPED VIDEO CLIP #{nth}")
        x1, x2, y1, y2 = row["x1"], row["x2"], row["y1"], row["y2"]
        video_crop = video_clip.fx(vfx.crop, x1=x1, x2=x2, y1=y1, y2=y2)
        
        vid_save_filename = f"{row['filename']}_trimmed_{trim_duration}sec_ts{ts}-te{te}_cropped_{x1}-{x2}-{y1}-{y2}.mp4" # [trim_duration_ts_te - crop_x1_x2_y1_y2]
        video_crop.write_videofile(f"{vid_path}{vid_save_filename}", fps=int(row["fps"]), codec="libx264", audio=False)
        video_crop.close()
        video_clip.close()
        

# Finally - Export those frames!

In [14]:
EXTRACT_FRAMES = True

for nth, row in df.iterrows():
    vid_filename = row["save_filename"]
    frames_filepaths = f"./data/images/{vid_filename.split('.')[0]}/"
    zfill_ = 5

    if EXTRACT_FRAMES:
        os.makedirs(frames_filepaths, exist_ok=True)
        video = cv2.VideoCapture(f"{vid_path}{vid_filename}")
        frame_counter = 0
        while True:
            ret, frame = video.read()
            if not ret:
                break
            if frame_counter % 1 == 0:
                cv2.imwrite(f"{frames_filepaths}/frame_{str(frame_counter).zfill(zfill_)}.jpg", frame)  
            frame_counter += 1
        video.release()